In [ ]:
 #prepare_environment.sh
    
echo "Creating Datastore/App Engine instance"
gcloud app create --region "us-central"


echo "Exporting GCLOUD_PROJECT"
export GCLOUD_PROJECT=$DEVSHELL_PROJECT_ID


echo "Creating virtual environment"
mkdir ~/venvs
virtualenv -p python3 ~/venvs/developingapps
source ~/venvs/developingapps/bin/activate

echo "Installing Python libraries"
pip install --upgrade pip
pip install -r requirements.txt

echo "Creating Datastore entities"
python add_entities.py

echo "Project ID: $DEVSHELL_PROJECT_ID"


In [ ]:
# Get the Bucket name from the
# GCLOUD_BUCKET environment variable
bucket_name = os.getenv('GCLOUD_BUCKET')

# Import the storage module
from google.cloud import storage

# Create a client for Cloud Storage
storage_client = storage.Client()

# Use the client to get the Cloud Storage bucket
bucket = storage_client.get_bucket(bucket_name)


In [ ]:
"""
Uploads a file to a given Cloud Storage bucket and returns the public url
to the new object.
"""
def upload_file(image_file, public):

    # Use the bucket to get a blob object
    blob = bucket.blob(image_file.filename)
    
    # Use the blob to upload the file
    blob.upload_from_string(
        image_file.read(),
        content_type=image_file.content_type)

    # Make the object public
    if public:
        blob.make_public()

    # Modify to return the blob's Public URL
    return blob.public_url



In [ ]:
"""
uploads file into google cloud storage
- call method to upload file (public=true)
- call datastore helper method to save question
"""
def save_question(data, image_file):

    # If there is an image file, then upload it
    # And assign the result to a new Datastore
    # property imageUrl
    # If there isn't, assign an empty string

    if image_file:
        data['imageUrl'] = str(
                  upload_file(image_file, True))
    else:
        data['imageUrl'] = u''

    data['correctAnswer'] = int(data['correctAnswer'])
    datastore.save_question(data)
    return

